In [7]:
import numpy as np
import tensorflow as tf

import scipy.misc
import glob
import sys


In [15]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr/255.0
        images.append(img_arr)

    return images

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)
    

In [16]:
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')


{'frog': 6, 'horse': 7, 'automobile': 1, 'bird': 2, 'ship': 8, 'airplane': 0, 'deer': 4, 'cat': 3, 'truck': 9, 'dog': 5}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [26]:
print(X_train[0].shape)
print(y_train[0])

(32, 32, 3)
6


In [27]:
def part1_model(features, labels, mode):
    input_layer = tf.reshape(features['x'], [-1, 32, 32, 3])
    conv1 = tf.layers.conv2d(inputs=input_layer,
                                 filters=32,
                                 kernel_size=[5, 5],
                                 padding='same',
                                 activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                   pool_size=[2, 2],
                                   strides=2)
    
    conv2 = tf.layers.conv2d(inputs=pool1,
                            filters=64,
                            kernel_size=[5, 5],
                            padding='same',
                            activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                   pool_size=[2, 2],
                                   strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
    
    dense = tf.layers.dense(input=pool2,
                            units=1024,
                            activation=tf.nn.relu)
    
    output_layer = tf.layers.dense(input=dense, units=10)
    
    predictions = {'classes': tf.argmax(input=output_layer, axis=1),
                   'prob': tf.nn.softmax(output_layer, name='softmax')}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    OH_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=OH_labels,
                                           logits=output_layer)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = GradientDescentOptimizer(learning_rate=.0001)
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode,
                                          loss=loss,
                                          train_op=train_op)
    
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels,
                                                       predictions=predictions['classes'])}
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=losss,
                                      eval_metric_ops=eval_metric_ops)


In [37]:
def main(unused_argv):
    cifar_classify = tf.estimator.Estimator(model_fn=part1_model,
                                            model_dir='/tmp/part1')

    tensors_log = {'probabilities': 'softmax'}
    hook = tf.train.LoggingTensorHook(tensors=tensors_log, every_n_iter=50)

    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x': X_train},
                                                        y=y_train,
                                                        batch_size=100,
                                                        num_epochs=None,
                                                        shuffle=True)
    
    cifar_classify.train(input_fn=train_input_fn,
                         steps=20000,
                         hooks=[hook])
    

In [38]:
main(1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


AttributeError: 'list' object has no attribute 'shape'